In [1]:
%matplotlib inline  

from os.path import isfile, join, exists
from os import listdir, makedirs, walk, remove, getlogin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from nilearn import image, masking
import scipy.stats as stats
import os
import numpy as np
import pickle

/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
projdir = '/mnt/bucket/labs/hasson/mai/projects/b2b_teaching'
    
run_list = ['teach1_epi_lesson1', 'teach2_epi_lesson2', 'teach2_epi_lesson3', 'teach3_epi_lesson4', 'teach3_epi_lesson5']

roi_list = ['julich_A1_TE1_both_bin20_3mm', 'v1_recent_25ss_3mm', 'sherlockAvg_fc_thr5_lSFG_3mm', 
            'sherlockAvg_fc_thr5_rSFG_3mm', 'sherlockAvg_fc_thr5_lTPJ_3mm', 'sherlockAvg_fc_thr5_rTPJ_3mm', 
            'sherlockAvg_fc_thr5_mpfc_3mm', 'sherlockAvg_fc_thr5_pmc_3mm']

In [3]:
# load data and get rois

# initialize an roi dict
roidict = {}
for roi in roi_list:
    roidict[roi] = {}

# grab data from all runs
for run in run_list:
    print('getting rois for ' + run)
    
    # load data
    datafile = join(projdir, 'teach_data', 'data', run + '_data.pk1')
    pkl_file = open(datafile, 'rb')
    datadict = pickle.load(pkl_file)
    pkl_file.close()

    # unmask data
    mask_nifti = image.load_img(datadict['mask'])
    data_unmask = masking.unmask(datadict['data'].T, mask_nifti)
    
    # mask by roi
    for roi in roi_list:
        roi_nifti = image.load_img(join(projdir, 'rois', roi + '.nii.gz'))
        roidata = masking.apply_mask(data_unmask, roi_nifti)
        roidict[roi][run] = roidata.T
        
print('\ndone!')


getting rois for teach1_epi_lesson1
getting rois for teach2_epi_lesson2
getting rois for teach2_epi_lesson3
getting rois for teach3_epi_lesson4
getting rois for teach3_epi_lesson5

done!


In [4]:
# save rois
for roi in roi_list:
    this_roidict = roidict[roi]
    
    savename = join(projdir, 'teach_data', 'data', 'roidata', roi + '_teachData.pk1')
    f = open(savename,"wb")
    pickle.dump(this_roidict,f)
    f.close()

print('done saving!')



done saving!


In [1]:
# make some figures
roi = roi_list[0]  
run = run_list[0]
roidict[roi]

for roi in roi_list:
    fig=plt.figure(figsize=(12, 4), dpi= 80, facecolor='w', edgecolor='k')

    for run in run_list:
        data = roidict[roi][run]
        data_mean = np.mean(data,axis=0)
        plt.plot(data_mean[1:100])
    #plt.plot(data[50,1:100])

    plt.title(roi)
    fig.show()

NameError: name 'roi_list' is not defined